### Restart and Run All

In [1]:
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine('sqlite:///c:\\ruby\\portlt\\db\\development.sqlite3')
conlt = engine.connect()

pd.set_option('display.float_format','{:,.2f}'.format)
pd.set_option('display.max_rows',None)

year = 2023
quarter = 1

today = date.today()
today

datetime.date(2023, 6, 19)

In [2]:
sql = '''
SELECT name,year,quarter,q_amt 
FROM epss 
WHERE (year = %s AND quarter <= %s)
OR (year = %s-1 AND quarter >= %s+1) 
ORDER BY year DESC, quarter DESC'''
sql = sql % (year, quarter, year, quarter)
dfc = pd.read_sql(sql, conlt)
dfc['Counter'] = 1
dfc_grp = dfc.groupby(['name'], as_index=False).sum()
dfc_grp = dfc_grp[dfc_grp['Counter'] == 4]
dfc_grp

,name,year,quarter,q_amt,Counter
0,ACE,8089,10,1234293,4
1,ADVANC,8089,10,26457375,4
3,AH,8089,10,1983003,4
4,AIE,8089,10,-151353,4
5,AIMIRT,8089,10,682646,4
6,AIT,8089,10,472255,4
7,AJ,8089,10,-306699,4
8,AMATA,8089,10,2279873,4
9,ANAN,8089,10,-74085,4
10,AOT,8089,10,-6473437,4


In [3]:
sql = '''
SELECT name,year,quarter,q_amt 
FROM epss 
WHERE (year = %s AND quarter <= %s-1) 
OR (year = %s-1 AND quarter >= %s) 
ORDER BY year DESC, quarter DESC'''
sql = sql % (year, quarter, year, quarter)
print(sql)


SELECT name,year,quarter,q_amt 
FROM epss 
WHERE (year = 2023 AND quarter <= 1-1) 
OR (year = 2023-1 AND quarter >= 1) 
ORDER BY year DESC, quarter DESC


In [4]:
dfp = pd.read_sql(sql, conlt)
dfp['Counter'] = 1
dfp_grp = dfp.groupby(['name'], as_index=False).sum()
dfp_grp = dfp_grp[dfp_grp['Counter'] == 4]
dfp_grp.shape

(232, 5)

In [5]:
dfm = pd.merge(dfc_grp, dfp_grp, on='name', suffixes=(['_c','_p']), how='inner')
dfm['inc_profit'] = dfm['q_amt_c']-dfm['q_amt_p']
dfm['Pct'] = round(dfm['inc_profit']/abs(dfm['q_amt_p'])*100,2)
dfm['year'] = year
dfm['quarter'] = 'Q'+str(quarter)
df_pct = dfm[['name','year','quarter','q_amt_c','q_amt_p','inc_profit','Pct']]
df_pct

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,Pct
0,ACE,2023,Q1,1234293,1281979,-47686,-3.72
1,ADVANC,2023,Q1,26457375,26011284,446091,1.71
2,AH,2023,Q1,1983003,1823648,159355,8.74
3,AIE,2023,Q1,-151353,-22132,-129221,-583.86
4,AIMIRT,2023,Q1,682646,688432,-5786,-0.84
5,AIT,2023,Q1,472255,541645,-69390,-12.81
6,AJ,2023,Q1,-306699,29609,-336308,"-1,135.83"
7,AMATA,2023,Q1,2279873,2341355,-61482,-2.63
8,ANAN,2023,Q1,-74085,-295785,221700,74.95
9,AOT,2023,Q1,-6473437,-11087867,4614430,41.62


In [6]:
sql = """
DELETE FROM qt_profits 
WHERE year = %s AND quarter = 'Q%s'"""
sql = sql % (year,quarter)
print(sql)


DELETE FROM qt_profits 
WHERE year = 2023 AND quarter = 'Q1'


In [7]:
rp = conlt.execute(sql)
rp.rowcount

179

In [8]:
sql = 'SELECT name, id FROM tickers'
tickers = pd.read_sql(sql, conlt)
tickers.shape

(401, 2)

In [9]:
df_ins = pd.merge(df_pct, tickers, on='name', how='inner')
df_ins.dtypes

name           object
year            int64
quarter        object
q_amt_c         int64
q_amt_p         int64
inc_profit      int64
Pct           float64
id              int64
dtype: object

In [10]:
rcds = df_ins.values.tolist()
len(rcds)

213

In [11]:
for rcd in rcds:
    print(rcd)

['ACE', 2023, 'Q1', 1234293, 1281979, -47686, -3.72, 698]
['ADVANC', 2023, 'Q1', 26457375, 26011284, 446091, 1.71, 6]
['AH', 2023, 'Q1', 1983003, 1823648, 159355, 8.74, 9]
['AIE', 2023, 'Q1', -151353, -22132, -129221, -583.86, 720]
['AIMIRT', 2023, 'Q1', 682646, 688432, -5786, -0.84, 669]
['AIT', 2023, 'Q1', 472255, 541645, -69390, -12.81, 11]
['AJ', 2023, 'Q1', -306699, 29609, -336308, -1135.83, 12]
['AMATA', 2023, 'Q1', 2279873, 2341355, -61482, -2.63, 21]
['ANAN', 2023, 'Q1', -74085, -295785, 221700, 74.95, 23]
['AOT', 2023, 'Q1', -6473437, -11087867, 4614430, 41.62, 24]
['AP', 2023, 'Q1', 5625945, 5877448, -251503, -4.28, 25]
['ASIAN', 2023, 'Q1', 780969, 978805, -197836, -20.21, 36]
['ASK', 2023, 'Q1', 1563900, 1512140, 51760, 3.42, 38]
['ASP', 2023, 'Q1', 460634, 479274, -18640, -3.89, 40]
['ASW', 2023, 'Q1', 1175597, 1118522, 57075, 5.1, 728]
['AWC', 2023, 'Q1', 4630796, 3853861, 776935, 20.16, 699]
['BA', 2023, 'Q1', -215102, -2110184, 1895082, 89.81, 45]
['BAM', 2023, 'Q1', 26

In [12]:
sql = """
INSERT INTO qt_profits (name, year, quarter, latest_amt, previous_amt, inc_amt, inc_pct, ticker_id) \
 VALUES (?, ?, ?, ?, ?, ?, ?, ?)"""
print(sql)


INSERT INTO qt_profits (name, year, quarter, latest_amt, previous_amt, inc_amt, inc_pct, ticker_id)  VALUES (?, ?, ?, ?, ?, ?, ?, ?)


In [13]:
for rcd in rcds:
    conlt.execute(sql, rcd)

### End of loop

In [14]:
cols = 'name year quarter q_amt_c q_amt_p inc_profit Pct'.split()

In [15]:
criteria_1 = df_ins.q_amt_c > 440000
df_ins.loc[criteria_1,cols].sort_values(['Pct'],ascending=[False]).head()

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,Pct
33,BKI,2023,Q1,3825890,-638378,4464268,699.31
37,CENTEL,2023,Q1,1070854,398082,672772,169.00
59,EGATIF,2023,Q1,878363,425689,452674,106.34
69,GPSC,2023,Q1,1695907,891449,804458,90.24
182,TIPCO,2023,Q1,506920,279818,227102,81.16


In [16]:
criteria_2 = df_ins.q_amt_p > 400000
df_ins.loc[criteria_2,cols].sort_values(['Pct'],ascending=[False]).head()

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,Pct
59,EGATIF,2023,Q1,878363,425689,452674,106.34
69,GPSC,2023,Q1,1695907,891449,804458,90.24
113,MINT,2023,Q1,7104207,4286373,2817834,65.74
210,WHAUP,2023,Q1,633231,454143,179088,39.43
175,TASCO,2023,Q1,3278554,2366511,912043,38.54


In [17]:
criteria_3 = df_ins.Pct > 10
df_ins.loc[criteria_3,cols].sort_values(['Pct'],ascending=[False]).head()

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,Pct
33,BKI,2023,Q1,3825890,-638378,4464268,699.31
172,SUPEREIF,2023,Q1,318028,73438,244590,333.06
37,CENTEL,2023,Q1,1070854,398082,672772,169.00
59,EGATIF,2023,Q1,878363,425689,452674,106.34
70,GRAMMY,2023,Q1,321728,166411,155317,93.33


In [18]:
df_ins_criteria = criteria_1 & criteria_2 & criteria_3
df_ins.loc[df_ins_criteria,cols].sort_values(['Pct'],ascending=[False]).head()

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,Pct
59,EGATIF,2023,Q1,878363,425689,452674,106.34
69,GPSC,2023,Q1,1695907,891449,804458,90.24
113,MINT,2023,Q1,7104207,4286373,2817834,65.74
210,WHAUP,2023,Q1,633231,454143,179088,39.43
175,TASCO,2023,Q1,3278554,2366511,912043,38.54
